# NEA Lightning Alert API Testing

This notebook demonstrates how to call the Singapore NEA Lightning Alert API, extract lightning coordinates, and display them on an interactive map.

## Features:
- Connect to NEA Lightning Alert API
- Parse JSON response data
- Extract lightning strike coordinates
- Display strikes on interactive Singapore map
- Visualize real-time lightning activity

## 1. Import Required Libraries

Import necessary libraries for API calls, data handling, and interactive mapping.

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import arrow
import folium
from folium import plugins
import time

# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries imported successfully!")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Setup API Configuration

Configure the NEA Lightning Alert API endpoint and parameters.

In [ ]:
# NEA Lightning Alert API Configuration - Updated to new endpoint
API_BASE_URL = "https://api-open.data.gov.sg/v2/real-time/api/weather"

# API parameters - Updated for new API format
API_PARAMS = {
    "api": "lightning",     # Required parameter set to 'lightning'
    "date": None           # Optional: YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS format
}

# Request headers - Optional API key for higher rate limits
HEADERS = {
    "User-Agent": "Lightning Detection System/1.0",
    "Accept": "application/json",
    # "x-api-key": "your-api-key-here"  # Uncomment and add your API key if available
}

# Singapore bounding box for map centering
SINGAPORE_BOUNDS = {
    "lat_min": 0.95,
    "lat_max": 1.75, 
    "lon_min": 103.27,
    "lon_max": 104.52,
    "center_lat": 1.3521,
    "center_lon": 103.8198
}

print("API configuration completed!")
print(f"API Endpoint: {API_BASE_URL}")
print(f"Required API parameter: api=lightning")
print(f"Singapore Center: ({SINGAPORE_BOUNDS['center_lat']}, {SINGAPORE_BOUNDS['center_lon']})")

## 3. Make API Call with JSON Parameters

Function to call the NEA Lightning Alert API and retrieve current lightning data.

1.691913210720644, 103.27386887665463
1.7448777400884117, 104.49982686163862
0.9502785290497732, 103.22328907766709
0.967136101860675, 104.52391248020412

In [ ]:
def fetch_lightning_data(date_filter=None):
    """
    Fetch lightning data from NEA API using the new v2 endpoint
    
    Args:
        date_filter: Optional date filter in YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS format
        
    Returns: JSON response or None if failed
    """
    try:
        # Prepare parameters for the new API
        params = {
            "api": "lightning"  # Required parameter
        }
        
        # Add optional date filter if provided
        if date_filter:
            params["date"] = date_filter
            print(f"Fetching lightning data for date: {date_filter}")
        else:
            print("Fetching latest lightning observations (no date filter)")
        
        # Make API call with updated endpoint
        print(f"Calling NEA Lightning API: {API_BASE_URL}")
        print(f"Parameters: {params}")
        
        response = requests.get(
            API_BASE_URL, 
            params=params,
            headers=HEADERS,
            timeout=10
        )
        
        print(f"Response Status Code: {response.status_code}")
        print(f"Full URL called: {response.url}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ API call successful!")
            print(f"Response received at: {datetime.now().strftime('%H:%M:%S')}")
            return data
        elif response.status_code == 400:
            print(f"❌ Bad Request (400): Invalid parameters")
            print(f"Response: {response.text}")
            return None
        elif response.status_code == 404:
            print(f"❌ Not Found (404): Weather data not found for the specified date")
            print(f"Response: {response.text}")
            return None
        else:
            print(f"❌ API call failed with status: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error: {e}")
        return None

# Test the API call with latest data (no date filter)
print("=== Testing API with latest data ===")
lightning_data = fetch_lightning_data()

# Store the response for later analysis
lightning_response = lightning_data

if lightning_data:
    print(f"\n📊 API Response Structure:")
    print(f"- Response Code: {lightning_data.get('code', 'N/A')}")
    print(f"- Error Message: {lightning_data.get('errorMsg', 'None')}")
    
    # Handle the new API response structure
    data_section = lightning_data.get('data', {})
    records = data_section.get('records', [])
    print(f"- Records Count: {len(records)}")
    
    # Check if we have actual lightning data
    total_readings = 0
    for record in records:
        item = record.get('item', {})
        readings = item.get('readings', [])
        total_readings += len(readings)
    
    print(f"- Total Lightning Readings: {total_readings}")
    
    if total_readings == 0:
        print("\n⚠️ No lightning activity detected in latest observation")
        print("💡 This is normal - lightning is not always present")
        print("💡 Try testing with a specific date when lightning was active")
else:
    print("No data received from API")

# Test with a specific date (example from today)
print(f"\n=== Testing API with today's date ===")
today_dt = arrow.get('2025-09-02T08:30:00+08:00').strftime('%Y-%m-%dT%H:%M:%S')
lightning_data_today = fetch_lightning_data(today_dt)

In [ ]:
lightning_data_today

## 4. Parse Lightning Alert Response

Parse the API response and handle the JSON structure to extract meaningful data.

In [ ]:
def parse_lightning_response(api_response):
    """
    Parse the NEA Lightning API response and extract relevant information
    Updated for new v2 API format that includes multiple time periods and pagination
    
    Args:
        api_response: JSON response from NEA API
        
    Returns:
        dict: Parsed lightning data with metadata and readings
    """
    if not api_response or api_response.get('code') != 0:
        print("❌ Invalid or error response from API")
        print(f"Response code: {api_response.get('code', 'Unknown') if api_response else 'No response'}")
        print(f"Error message: {api_response.get('errorMsg', 'No error message') if api_response else 'No response'}")
        return None
    
    # Initialize parsed data structure
    parsed_data = {
        'timestamp': datetime.now().isoformat(),
        'api_status': api_response.get('code'),
        'error_message': api_response.get('errorMsg', ''),
        'pagination_token': api_response.get('data', {}).get('paginationToken', None),
        'records': [],
        'total_strikes': 0,
        'records_with_lightning': 0,
        'time_range': {
            'earliest': None,
            'latest': None
        }
    }
    
    # Extract records from data section
    data_section = api_response.get('data', {})
    records = data_section.get('records', [])
    
    print(f"📊 Processing {len(records)} time period records from API...")
    
    # Track time range
    all_datetimes = []
    
    for record in records:
        record_datetime = record.get('datetime')
        updated_timestamp = record.get('updatedTimestamp')
        
        if record_datetime:
            all_datetimes.append(record_datetime)
        
        # Extract item data
        item = record.get('item', {})
        is_station_data = item.get('isStationData', False)
        observation_type = item.get('type', 'unknown')
        readings = item.get('readings', [])
        
        # Create record data structure
        record_data = {
            'datetime': record_datetime,
            'updated_timestamp': updated_timestamp,
            'is_station_data': is_station_data,
            'observation_type': observation_type,
            'readings_count': len(readings),
            'readings': []
        }
        
        # Process lightning readings for this time period
        if readings:
            parsed_data['records_with_lightning'] += 1
            print(f"⚡ Found {len(readings)} lightning strikes at {record_datetime}")
            
            for reading in readings:
                location = reading.get('location', {})
                
                # Parse coordinates (they come as strings from API)
                try:
                    latitude = float(location.get('latitude', 0))
                    longitude = float(location.get('longitude', 0))
                except (ValueError, TypeError):
                    print(f"⚠️ Invalid coordinates in reading: {location}")
                    continue
                
                reading_data = {
                    'latitude': latitude,
                    'longitude': longitude,
                    'type': reading.get('type', 'Unknown'),
                    'description': reading.get('text', 'Unknown'),
                    'datetime': reading.get('datetime'),
                    'record_datetime': record_datetime  # Time period this reading belongs to
                }
                
                record_data['readings'].append(reading_data)
                parsed_data['total_strikes'] += 1
        else:
            # Record with no lightning activity
            print(f"📝 No lightning activity at {record_datetime}")
        
        parsed_data['records'].append(record_data)
    
    # Determine time range
    if all_datetimes:
        parsed_data['time_range']['earliest'] = min(all_datetimes)
        parsed_data['time_range']['latest'] = max(all_datetimes)
    
    # Add pagination info
    if parsed_data['pagination_token']:
        print(f"📄 Pagination token available for retrieving more data")
    
    print(f"✅ Parsing complete:")
    print(f"   - Time periods processed: {len(records)}")
    print(f"   - Periods with lightning: {parsed_data['records_with_lightning']}")
    print(f"   - Total lightning strikes: {parsed_data['total_strikes']}")
    if parsed_data['time_range']['earliest']:
        print(f"   - Time range: {parsed_data['time_range']['earliest']} to {parsed_data['time_range']['latest']}")
    
    return parsed_data

# Parse the lightning data
print("🔍 Parsing lightning data from API response...")

if lightning_data:
    parsed_lightning = parse_lightning_response(lightning_data_today)
    
    if parsed_lightning:
        print("\n✅ Lightning data parsed successfully!")
        print(f"📊 Detailed Summary:")
        print(f"- API Status Code: {parsed_lightning['api_status']}")
        print(f"- Total Time Periods: {len(parsed_lightning['records'])}")
        print(f"- Periods with Lightning: {parsed_lightning['records_with_lightning']}")
        print(f"- Total Lightning Strikes: {parsed_lightning['total_strikes']}")
        print(f"- Data Parsed at: {parsed_lightning['timestamp']}")
        
        # Show time range
        if parsed_lightning['time_range']['earliest']:
            print(f"- Time Range: {parsed_lightning['time_range']['earliest']} to {parsed_lightning['time_range']['latest']}")
        
        # Show pagination info
        if parsed_lightning['pagination_token']:
            print(f"- Pagination Available: Yes (token present)")
        else:
            print(f"- Pagination Available: No")
        
        # Display sample lightning strike data if available
        if parsed_lightning['total_strikes'] > 0:
            print(f"\n🌩️ Sample Lightning Strike Details:")
            
            # Find first record with readings
            sample_reading = None
            for record in parsed_lightning['records']:
                if record['readings']:
                    sample_reading = record['readings'][0]
                    sample_record_time = record['datetime']
                    break
            
            if sample_reading:
                print(f"- Record Time Period: {sample_record_time}")
                print(f"- Strike Location: ({sample_reading['latitude']:.4f}, {sample_reading['longitude']:.4f})")
                print(f"- Lightning Type: {sample_reading['type']} ({sample_reading['description']})")
                print(f"- Strike Timestamp: {sample_reading['datetime']}")
                
                # Show how many strikes in the most active period
                max_strikes_record = max(parsed_lightning['records'], key=lambda x: len(x['readings']))
                if max_strikes_record['readings']:
                    print(f"- Most Active Period: {max_strikes_record['datetime']} ({len(max_strikes_record['readings'])} strikes)")
        else:
            print(f"\n📝 No lightning strikes detected in the current data")
            print(f"💡 This is normal when weather conditions are clear")
            print(f"💡 Try testing during thunderstorm activity or with historical dates")
            
    else:
        print("❌ Failed to parse lightning data")
        parsed_lightning = {'records': [], 'total_strikes': 0, 'records_with_lightning': 0}
else:
    print("❌ No lightning data available to parse")
    parsed_lightning = {'records': [], 'total_strikes': 0, 'records_with_lightning': 0}

## 5. Extract Coordinates Data

Extract and organize lightning strike coordinates into a structured format for mapping.

In [ ]:
def extract_coordinates(parsed_data):
    """
    Extract coordinates from parsed lightning data and create a pandas DataFrame
    
    Args:
        parsed_data: Parsed lightning data from parse_lightning_response()
        
    Returns:
        pandas.DataFrame: DataFrame with lightning coordinates and metadata
    """
    coordinates_list = []
    
    if not parsed_data or not parsed_data.get('records'):
        print("⚠️ No lightning data available for coordinate extraction")
        return pd.DataFrame(columns=['latitude', 'longitude', 'type', 'description', 'datetime', 'record_datetime'])
    
    for record in parsed_data['records']:
        record_time = record.get('datetime', 'Unknown')
        
        for reading in record.get('readings', []):
            coord_data = {
                'latitude': reading['latitude'],
                'longitude': reading['longitude'],
                'type': reading['type'],
                'description': reading['description'],
                'datetime': reading['datetime'],
                'record_datetime': record_time
            }
            coordinates_list.append(coord_data)
    
    # Create DataFrame
    df = pd.DataFrame(coordinates_list)
    
    # Validate coordinates are within Singapore bounds
    if not df.empty:
        valid_coords = (
            (df['latitude'] >= SINGAPORE_BOUNDS['lat_min']) & 
            (df['latitude'] <= SINGAPORE_BOUNDS['lat_max']) &
            (df['longitude'] >= SINGAPORE_BOUNDS['lon_min']) & 
            (df['longitude'] <= SINGAPORE_BOUNDS['lon_max'])
        )
        
        print(f"📍 Coordinate Summary:")
        print(f"- Total coordinates extracted: {len(df)}")
        print(f"- Coordinates within Singapore bounds: {valid_coords.sum()}")
        print(f"- Lightning types: {df['type'].unique().tolist()}")
        
        if valid_coords.sum() < len(df):
            print(f"⚠️ {len(df) - valid_coords.sum()} coordinates outside Singapore bounds")
        
        return df
    else:
        print("❌ No valid coordinates found")
        return df

# Extract coordinates from parsed data
coordinates_df = extract_coordinates(parsed_lightning)

# Display coordinate data
if not coordinates_df.empty:
    print(f"\n📋 Coordinates DataFrame:")
    print(coordinates_df.head())
    
    print(f"\n📊 Coordinate Statistics:")
    print(f"- Latitude range: {coordinates_df['latitude'].min():.4f} to {coordinates_df['latitude'].max():.4f}")
    print(f"- Longitude range: {coordinates_df['longitude'].min():.4f} to {coordinates_df['longitude'].max():.4f}")
else:
    print("📍 No lightning coordinates available for mapping")
    # Create sample data for demonstration if no real data
    sample_data = [
        {'latitude': 1.3521, 'longitude': 103.8198, 'type': 'C', 'description': 'Sample - Cloud to Cloud', 'datetime': datetime.now().isoformat()},
        {'latitude': 1.3000, 'longitude': 103.8500, 'type': 'C', 'description': 'Sample - Cloud to Cloud', 'datetime': datetime.now().isoformat()}
    ]
    coordinates_df = pd.DataFrame(sample_data)
    print("📍 Using sample coordinates for demonstration")

## 6. Create Interactive Map

Create an interactive map of Singapore with lightning strike locations.

In [ ]:
def create_singapore_map(coordinates_df):
    """
    Create an interactive map of Singapore with lightning strikes
    
    Args:
        coordinates_df: DataFrame with lightning coordinates and metadata
        
    Returns:
        folium.Map: Interactive map with lightning markers
    """
    # Singapore center coordinates
    singapore_center = [1.3521, 103.8198]
    
    # Create base map
    lightning_map = folium.Map(
        location=singapore_center,
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Add Singapore boundary rectangle
    folium.Rectangle(
        bounds=[[SINGAPORE_BOUNDS['lat_min'], SINGAPORE_BOUNDS['lon_min']],
                [SINGAPORE_BOUNDS['lat_max'], SINGAPORE_BOUNDS['lon_max']]],
        color='blue',
        fill=False,
        weight=2,
        popup='Singapore Boundary'
    ).add_to(lightning_map)
    
    # Add lightning markers if data exists
    if not coordinates_df.empty:
        for idx, row in coordinates_df.iterrows():
            # Different colors for different lightning types
            color = 'red' if row['type'] == 'G' else 'orange'
            icon_symbol = '⚡' if row['type'] == 'G' else '☁️'
            
            # Create popup text
            popup_text = f"""
            <b>Lightning Strike</b><br>
            Type: {row['type']} ({row['description']})<br>
            Time: {row['datetime']}<br>
            Coordinates: {row['latitude']:.4f}, {row['longitude']:.4f}
            """
            
            # Add marker
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=folium.Popup(popup_text, max_width=300),
                tooltip=f"{icon_symbol} {row['type']} - {row['datetime']}",
                icon=folium.Icon(color=color, icon='flash', prefix='fa')
            ).add_to(lightning_map)
    
    # Add marker for Singapore center
    folium.Marker(
        location=singapore_center,
        popup='Singapore Center',
        tooltip='Singapore Center',
        icon=folium.Icon(color='green', icon='home', prefix='fa')
    ).add_to(lightning_map)
    
    return lightning_map

# Create the map
print("🗺️ Creating interactive Singapore map...")
singapore_map = create_singapore_map(coordinates_df)

# Display map summary
marker_count = len(coordinates_df) if not coordinates_df.empty else 0
print(f"✅ Map created successfully!")
print(f"📍 Markers added: {marker_count} lightning strikes")
print(f"🌍 Map center: Singapore (1.3521, 103.8198)")

# Display the map
singapore_map

## 7. Add Distance Analysis

Calculate distances from lightning strikes to key locations and analyze coverage areas.

In [ ]:
from geopy.distance import geodesic
import numpy as np

def calculate_lightning_distances(coordinates_df, reference_points=None):
    """
    Calculate distances from lightning strikes to reference points
    
    Args:
        coordinates_df: DataFrame with lightning coordinates
        reference_points: Dict of reference locations {name: (lat, lon)}
        
    Returns:
        dict: Analysis results with distances and statistics
    """
    if reference_points is None:
        # Default Singapore reference points
        reference_points = {
            'Singapore Center': (1.3521, 103.8198),
            'Changi Airport': (1.3644, 103.9915),
            'Marina Bay': (1.2966, 103.8764),
            'Jurong East': (1.3329, 103.7436),
            'Woodlands': (1.4382, 103.7890)
        }
    
    analysis_results = {}
    
    if coordinates_df.empty:
        print("⚠️ No lightning data available for distance analysis")
        return analysis_results
    
    # Calculate distances for each reference point
    for location_name, (ref_lat, ref_lon) in reference_points.items():
        distances = []
        
        for idx, row in coordinates_df.iterrows():
            lightning_point = (row['latitude'], row['longitude'])
            reference_point = (ref_lat, ref_lon)
            
            # Calculate distance in kilometers
            distance_km = geodesic(lightning_point, reference_point).kilometers
            distances.append(distance_km)
        
        # Calculate statistics
        distances_array = np.array(distances)
        stats = {
            'min_distance': distances_array.min(),
            'max_distance': distances_array.max(),
            'mean_distance': distances_array.mean(),
            'median_distance': np.median(distances_array),
            'strikes_within_8km': (distances_array <= 8).sum(),
            'strikes_within_15km': (distances_array <= 15).sum(),
            'total_strikes': len(distances_array)
        }
        
        analysis_results[location_name] = {
            'distances': distances,
            'statistics': stats,
            'coordinates': (ref_lat, ref_lon)
        }
    
    return analysis_results

def create_enhanced_map_with_distances(coordinates_df, analysis_results):
    """
    Create enhanced map with distance circles and analysis
    """
    # Create base map
    enhanced_map = create_singapore_map(coordinates_df)
    
    # Add reference points and distance circles
    for location_name, data in analysis_results.items():
        ref_lat, ref_lon = data['coordinates']
        stats = data['statistics']
        
        # Add reference point marker
        folium.Marker(
            location=[ref_lat, ref_lon],
            popup=f"""
            <b>{location_name}</b><br>
            Lightning strikes within 8km: {stats['strikes_within_8km']}<br>
            Closest strike: {stats['min_distance']:.2f} km<br>
            Average distance: {stats['mean_distance']:.2f} km
            """,
            tooltip=f"📍 {location_name}",
            icon=folium.Icon(color='blue', icon='building', prefix='fa')
        ).add_to(enhanced_map)
        
        # Add 8km safety circle (outdoor activity halt zone)
        folium.Circle(
            location=[ref_lat, ref_lon],
            radius=8000,  # 8km in meters
            color='red',
            fill=True,
            fillOpacity=0.1,
            weight=2,
            popup=f'{location_name} - 8km Safety Zone'
        ).add_to(enhanced_map)
        
        # Add 15km monitoring circle
        folium.Circle(
            location=[ref_lat, ref_lon],
            radius=15000,  # 15km in meters
            color='orange',
            fill=False,
            weight=1,
            popup=f'{location_name} - 15km Monitoring Zone'
        ).add_to(enhanced_map)
    
    return enhanced_map

# Perform distance analysis
print("📏 Calculating distances from lightning strikes to key locations...")
distance_analysis = calculate_lightning_distances(coordinates_df)

# Display analysis results
if distance_analysis:
    print(f"\n📊 Lightning Distance Analysis Results:")
    print(f"{'Location':<20} {'Within 8km':<12} {'Within 15km':<13} {'Closest (km)':<13} {'Average (km)'}")
    print("-" * 75)
    
    for location, data in distance_analysis.items():
        stats = data['statistics']
        print(f"{location:<20} {stats['strikes_within_8km']:<12} {stats['strikes_within_15km']:<13} "
              f"{stats['min_distance']:<13.2f} {stats['mean_distance']:.2f}")
    
    # Create enhanced map with distance analysis
    print(f"\n🗺️ Creating enhanced map with distance circles...")
    enhanced_map = create_enhanced_map_with_distances(coordinates_df, distance_analysis)
    
    print(f"✅ Enhanced map created with:")
    print(f"   - {len(distance_analysis)} reference locations")
    print(f"   - 8km safety zones (red circles)")
    print(f"   - 15km monitoring zones (orange circles)")
    
    enhanced_map
else:
    print("❌ No distance analysis results available")
    singapore_map

## 8. Export and Summary

Export the results and provide a comprehensive summary of the lightning data analysis.

In [ ]:
def export_lightning_data(coordinates_df, distance_analysis, output_dir="../data"):
    """
    Export lightning data and analysis results to files
    
    Args:
        coordinates_df: DataFrame with lightning coordinates
        distance_analysis: Distance analysis results
        output_dir: Directory to save export files
    """
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Export coordinates to CSV
    if not coordinates_df.empty:
        csv_filename = f"{output_dir}/lightning_coordinates_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        coordinates_df.to_csv(csv_filename, index=False)
        print(f"📄 Coordinates exported to: {csv_filename}")
    
    # Export distance analysis to JSON
    if distance_analysis:
        import json
        
        # Convert numpy arrays to lists for JSON serialization
        exportable_analysis = {}
        for location, data in distance_analysis.items():
            exportable_analysis[location] = {
                'statistics': data['statistics'],
                'coordinates': data['coordinates'],
                'distance_count': len(data['distances'])
            }
        
        json_filename = f"{output_dir}/lightning_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(json_filename, 'w') as f:
            json.dump(exportable_analysis, f, indent=2, default=str)
        print(f"📊 Analysis results exported to: {json_filename}")
    
    return output_dir

def generate_summary_report(coordinates_df, distance_analysis, api_response_data):
    """
    Generate a comprehensive summary report
    """
    print("="*80)
    print("⚡ SINGAPORE LIGHTNING DETECTION - ANALYSIS SUMMARY")
    print("="*80)
    
    # API Status
    print(f"\n🌐 API Connection Status:")
    if api_response_data:
        print(f"   ✅ Successfully connected to NEA Lightning Alert API v2")
        print(f"   📡 API Endpoint: {API_BASE_URL}")
        print(f"   🔗 Using parameter: api=lightning")
        
        # Check API response code
        api_code = api_response_data.get('code', 'Unknown')
        error_msg = api_response_data.get('errorMsg', '')
        
        if api_code == 0:
            print(f"   ✅ API Response: Success (code: {api_code})")
        else:
            print(f"   ⚠️ API Response: Code {api_code}, Error: {error_msg}")
    else:
        print(f"   ❌ API connection failed or no data available")
    
    # Data Summary
    print(f"\n📊 Lightning Data Summary:")
    if not coordinates_df.empty:
        print(f"   📍 Total lightning strikes detected: {len(coordinates_df)}")
        print(f"   🌩️ Lightning types found: {', '.join(coordinates_df['type'].unique())}")
        print(f"   📅 Time range: {coordinates_df['datetime'].min()} to {coordinates_df['datetime'].max()}")
        print(f"   🗺️ Geographic coverage:")
        print(f"      - Latitude: {coordinates_df['latitude'].min():.4f}° to {coordinates_df['latitude'].max():.4f}°")
        print(f"      - Longitude: {coordinates_df['longitude'].min():.4f}° to {coordinates_df['longitude'].max():.4f}°")
    else:
        print(f"   ⚠️ No lightning data available (this is normal when there's no active lightning)")
        print(f"   💡 Lightning detection depends on actual weather conditions")
        print(f"   💡 Try again during thunderstorm activity or test with historical dates")
    
    # Safety Analysis
    print(f"\n🚨 Safety Alert Analysis:")
    if distance_analysis:
        total_alerts_8km = sum(data['statistics']['strikes_within_8km'] for data in distance_analysis.values())
        total_alerts_15km = sum(data['statistics']['strikes_within_15km'] for data in distance_analysis.values())
        
        print(f"   🔴 High Alert (8km zone): {total_alerts_8km} location-strikes")
        print(f"   🟡 Medium Alert (15km zone): {total_alerts_15km} location-strikes")
        
        if total_alerts_8km > 0:
            # Find most affected locations
            most_affected = max(distance_analysis.items(), 
                              key=lambda x: x[1]['statistics']['strikes_within_8km'])
            
            print(f"   📍 Most affected location: {most_affected[0]}")
            print(f"      - Strikes within 8km: {most_affected[1]['statistics']['strikes_within_8km']}")
            print(f"      - Closest strike: {most_affected[1]['statistics']['min_distance']:.2f} km")
        else:
            print(f"   ✅ No lightning strikes within 8km of monitored locations")
    
    # API Documentation Reference
    print(f"\n📚 API Information:")
    print(f"   🔗 Endpoint: https://api-open.data.gov.sg/v2/real-time/api/weather?api=lightning")
    print(f"   📖 Documentation: Check README.md for complete API details")
    print(f"   🔑 Optional: Add x-api-key header for higher rate limits")
    print(f"   📅 Date filters: Use YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS format")
    
    # Recommendations
    print(f"\n💡 Recommendations:")
    if not coordinates_df.empty and distance_analysis:
        high_risk_locations = [name for name, data in distance_analysis.items() 
                             if data['statistics']['strikes_within_8km'] > 0]
        
        if high_risk_locations:
            print(f"   🚫 IMMEDIATE ACTION: Halt outdoor activities at:")
            for location in high_risk_locations:
                print(f"      - {location}")
            print(f"   ⏰ Wait 30 minutes after last lightning before resuming activities")
        else:
            print(f"   ✅ No immediate lightning threats within 8km of monitored locations")
            print(f"   👁️ Continue monitoring for approaching storms")
    else:
        print(f"   📱 API connection working - ready for real-time monitoring")
        print(f"   🔧 Test during thunderstorm activity to verify alert system")
        print(f"   💾 System ready for integration with hardware alerts")
    
    print(f"\n🔄 Next Steps:")
    print(f"   1. Set up automated polling every 30 seconds during storms")
    print(f"   2. Configure alert thresholds based on your requirements")
    print(f"   3. Integrate with your hardware alert system")
    print(f"   4. Test with historical dates when lightning was active")
    print(f"   5. Add x-api-key header if you have higher rate limit needs")
    
    print("="*80)

# Export data
print("💾 Exporting lightning data and analysis results...")
export_dir = export_lightning_data(coordinates_df, distance_analysis)

# Generate comprehensive summary
generate_summary_report(coordinates_df, distance_analysis, lightning_response)

# Final instructions
print(f"\n🎯 NEXT STEPS FOR DEVELOPMENT:")
print(f"1. ✅ API updated to new v2 endpoint with correct parameters")
print(f"2. 🔧 Modify reference points to match your customer locations")
print(f"3. ⚙️ Adjust alert thresholds (8km/15km) based on requirements")
print(f"4. 🔗 Integrate this code into your main lightning alert application")
print(f"5. 🚨 Set up automated alerts using the distance analysis functions")
print(f"6. 🔑 Consider adding API key for higher rate limits")

print(f"\n📁 Files created in this session:")
print(f"   - Interactive maps displayed in notebook")
print(f"   - Exported data files in: {export_dir}")
print(f"   - Analysis functions ready for integration")

print(f"\n🌩️ API Testing Notes:")
print(f"   - No lightning data = Normal when weather is clear")
print(f"   - Test during thunderstorms for real lightning data")
print(f"   - Use date filters to test with historical storm data")
print(f"   - API returns latest observation when no date specified")

print(f"\n🚀 Ready to build your Singapore Lightning Alert System!")